daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
from spx_data_update import UpdateSP500Data
import numpy as np
import pandas as pd
import nest_asyncio
from time import time
import plistlib
file_name = UpdateSP500Data.DATA_BASE_PATH / 'config.plist'
f = open(str(file_name), 'rb')
pl = plistlib.load(f)
illiquid_dict = pl['illiquid_equity']
illiquid_equity = sum(illiquid_dict.values()) * 0.5
illiquid_equity

531500.0

In [5]:
before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [6]:
z_score = np.array([-0.5, -1, -1.5])
leverage = np.array(range(1, 3, 1))
num_expiries = 3

after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 70.5
elapsed: 84.7
2666.46
21.92
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2732593.1364', currency='BASE', modelCode='')]


In [7]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [9]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-0.5,-1.0,-1.5
2018.12.21,2620,2575,2545
2019.01.18,2570,2485,2430
2019.02.15,2525,2400,2320


,-0.5,-1.0,-1.5
2018.12.21,31.45,11.90,13.20
2019.01.18,34.70,19.95,14.00
2019.02.15,49.85,23.20,18.25


,-0.5,-1.0,-1.5
2018.12.21,-1.0/-1.0,11.7/12.1,-1.0/-1.0
2019.01.18,34.4/35.0,19.7/20.2,13.8/14.2
2019.02.15,-1.0/-1.0,22.9/23.5,-1.0/-1.0


In [10]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-0.5,-1.0,-1.5
2018.12.21,-1.74%,-3.43%,-4.56%
2019.01.18,-3.62%,-6.81%,-8.87%
2019.02.15,-5.31%,-9.99%,-12.99%


In [11]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-0.5,-1.0,-1.5
2018.12.21,"$40,885","$15,470","$17,160"
2019.01.18,"$45,110","$25,935","$19,600"
2019.02.15,"$64,805","$32,480","$25,550"


,-0.5,-1.0,-1.5
2018.12.21,"$78,625","$29,750","$34,320"
2019.01.18,"$90,220","$53,865","$37,800"
2019.02.15,"$129,610","$64,960","$51,100"


In [12]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-0.5,-1.0,-1.5
2018.12.21,1.18%,0.45%,0.50%
2019.01.18,1.30%,0.75%,0.53%
2019.02.15,1.87%,0.87%,0.68%


,-0.5,-1.0,-1.5
2018.12.21,2.36%,0.89%,0.99%
2019.01.18,2.60%,1.50%,1.05%
2019.02.15,3.74%,1.74%,1.37%


In [13]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,13,13,13
2019.01.18,13,13,14
2019.02.15,13,14,14


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,25,25,26
2019.01.18,26,27,27
2019.02.15,26,28,28


In [14]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-0.5,-1.0,-1.5
2018.12.21,"$673,767","$589,852","$552,542"
2019.01.18,"$612,992","$483,317","$435,165"
2019.02.15,"$574,187","$406,045","$350,350"


,-0.5,-1.0,-1.5
2018.12.21,"$1,295,705","$1,134,330","$1,105,083"
2019.01.18,"$1,225,983","$1,003,811","$839,246"
2019.02.15,"$1,148,373","$812,090","$700,700"


In [15]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$1,295,705","$1,134,330","$1,105,083"
2019.01.18,"$1,225,983","$1,003,811","$839,246"
2019.02.15,"$1,148,373","$812,090","$700,700"


In [16]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,"$3,406,000","$3,347,500","$3,308,500"
2019.01.18,"$3,341,000","$3,230,500","$3,402,000"
2019.02.15,"$3,282,500","$3,360,000","$3,248,000"


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$6,550,000","$6,437,500","$6,617,000"
2019.01.18,"$6,682,000","$6,709,500","$6,561,000"
2019.02.15,"$6,565,000","$6,720,000","$6,496,000"


In [17]:
capital_at_risk

3264093.1364

In [18]:
illiquid_equity

531500.0